## Deliverable 2. Create a Customer Travel Destinations Map.

In [38]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

# place = gmaps.find_place(input="1600 Amphitheatre Parkway, Mountain View, CA", input_type="textquery")
# place

In [39]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_vacation.csv")
city_data_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,2,Aksu,CN,82.42,clear sky,41.12,80.26,Pudong Holiday Hotel
1,7,Touros,BR,80.60,broken clouds,-5.20,-35.46,INN NEW HORIZON
2,10,Morehead,US,75.20,clear sky,37.27,-87.18,CCI Express Inn
3,11,Port Elizabeth,ZA,84.20,clear sky,-33.92,25.57,39 On Nile Guest House
4,16,Northam,GB,82.40,few clouds,51.03,-4.22,Durrant House Hotel
...,...,...,...,...,...,...,...,...
347,695,Kostomuksha,RU,79.27,overcast clouds,64.57,30.58,Apartments on Lenina 17
348,700,Sabratah,LY,85.15,clear sky,32.79,12.49,مقبرة المجاهديه
349,704,Wuwei,CN,75.00,overcast clouds,37.93,102.64,Thank Inn Chain Hotel gansu wuwei liangzhou di...
350,705,Chake Chake,TZ,78.87,few clouds,-5.25,39.77,Hotel Archipelago


In [40]:
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Max Temp               float64
Current Description     object
Lat                    float64
Lng                    float64
Hotel Name              object
dtype: object

In [41]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

In [42]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
temp_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

temp_cities_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,10,Morehead,US,75.20,clear sky,37.27,-87.18,CCI Express Inn
7,22,Atuona,PF,79.72,clear sky,-9.80,-139.03,Villa Enata
8,23,Kapaa,US,73.40,heavy intensity rain,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
10,28,Arraial Do Cabo,BR,78.80,clear sky,-22.97,-42.03,Pousada Porto Praia
11,29,Luganville,VU,77.72,light rain,-15.53,167.17,The Espiritu Hotel


In [43]:
# 4a. Determine if there are any empty rows.
if temp_cities_df.shape[0]==0:
    print('DF is empty')
else:
    print("DF is not empty")

DF is not empty


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [46]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = temp_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Morehead,US,75.20,clear sky,37.27,-87.18,
7,Atuona,PF,79.72,clear sky,-9.80,-139.03,
8,Kapaa,US,73.40,heavy intensity rain,22.08,-159.32,
10,Arraial Do Cabo,BR,78.80,clear sky,-22.97,-42.03,
11,Luganville,VU,77.72,light rain,-15.53,167.17,


In [47]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    Lat = row["Lat"]
    Lng = row["Lng"]

    params["location"]= f"{Lat},{Lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found")
        

Hotel not found


In [48]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(axis=0,how='any')


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Morehead,US,75.20,clear sky,37.27,-87.18,CCI Express Inn
7,Atuona,PF,79.72,clear sky,-9.80,-139.03,Villa Enata
8,Kapaa,US,73.40,heavy intensity rain,22.08,-159.32,Sheraton Kauai Coconut Beach Resort
10,Arraial Do Cabo,BR,78.80,clear sky,-22.97,-42.03,Pousada Porto Praia
11,Luganville,VU,77.72,light rain,-15.53,167.17,The Espiritu Hotel
...,...,...,...,...,...,...,...
346,Klyuchi,RU,76.15,broken clouds,52.27,79.17,Gostinitsa Luch
347,Kostomuksha,RU,79.27,overcast clouds,64.57,30.58,U Mikhailicha
349,Wuwei,CN,75.00,overcast clouds,37.93,102.64,Thank Inn Chain Hotel gansu wuwei liangzhou di...
350,Chake Chake,TZ,78.87,few clouds,-5.25,39.77,Hotel Archipelago


In [50]:
# 8a. Create the output File (CSV)
output_data_file= "vaca_hotel.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [51]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [52]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))